<a href="https://colab.research.google.com/github/OneFineStarstuff/TheOneEverAfter/blob/main/_Model_Evaluation_and_Further_Steps_in_Galaxy_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install optuna

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from torchvision.transforms import functional as F
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import optuna

# Placeholder for loading your dataset
train_images = torch.randn(100, 3, 64, 64)  # Example: 100 images, 3 channels, 64x64 resolution
train_labels = torch.randint(0, 5, (100,))  # Example: 100 labels, 5 classes
test_images = torch.randn(20, 3, 64, 64)   # Example: 20 test images
test_labels = torch.randint(0, 5, (20,))   # Example: 20 test labels

# Custom transform function
def tensor_transform(image):
    image = F.rotate(image, angle=30)
    if torch.rand(1) < 0.5:
        image = F.hflip(image)
    if torch.rand(1) < 0.5:
        image = F.vflip(image)
    image = F.resized_crop(image, top=0, left=0, height=64, width=64, size=(64, 64))
    return image

# Custom Dataset class
class GalaxyDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Improved CNN with Pre-trained Model
class GalaxyCNN(nn.Module):
    def __init__(self):
        super(GalaxyCNN, self).__init__()
        self.resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 5)  # Assuming 5 classes

        # Freeze early layers
        for param in self.resnet.parameters():
            param.requires_grad = False

        # Unfreeze and train only the final layers
        for param in self.resnet.fc.parameters():
            param.requires_grad = True

    def forward(self, x):
        x = self.resnet(x)
        return x

# Hyperparameter tuning with Optuna
def objective(trial):
    # Suggest hyperparameters
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_int('batch_size', 16, 128)

    # Create model, loss function, and optimizer
    model = GalaxyCNN()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.resnet.fc.parameters(), lr=lr)

    # Create DataLoader with suggested batch size
    train_dataset = GalaxyDataset(train_images, train_labels, transform=tensor_transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Training loop (simplified)
    model.train()
    for epoch in range(10):  # Example: 10 epochs
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Here you should evaluate model performance and return it
    # For simplicity, returning a dummy value (loss.item())
    return loss.item()

# Run Optuna optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

print("Best hyperparameters: ", study.best_params)

# Initialize model, loss function, and optimizer with best hyperparameters
best_lr = study.best_params['lr']
best_batch_size = study.best_params['batch_size']

model = GalaxyCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.resnet.fc.parameters(), lr=best_lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

train_dataset = GalaxyDataset(train_images, train_labels, transform=tensor_transform)
train_loader = DataLoader(train_dataset, batch_size=best_batch_size, shuffle=True)
test_dataset = GalaxyDataset(test_images, test_labels)
test_loader = DataLoader(test_dataset, batch_size=best_batch_size, shuffle=False)

# Training loop with scheduler
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    scheduler.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Evaluation
model.eval()
test_predictions = []
test_labels_list = []

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_predictions.extend(predicted.numpy())
        test_labels_list.extend(labels.numpy())

accuracy = accuracy_score(test_labels_list, test_predictions)
precision = precision_score(test_labels_list, test_predictions, average='weighted', zero_division=1)
recall = recall_score(test_labels_list, test_predictions, average='weighted')
f1 = f1_score(test_labels_list, test_predictions, average='weighted')

print(f"Test Accuracy: {accuracy:.2f}")
print(f"Test Precision: {precision:.2f}")
print(f"Test Recall: {recall:.2f}")
print(f"Test F1 Score: {f1:.2f}")